In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Nagarro - Airline Data").master("local[*]").getOrCreate()

24/03/28 00:01:45 WARN Utils: Your hostname, ajith-Lenovo-G50-80 resolves to a loopback address: 127.0.1.1; using 192.168.1.40 instead (on interface wlp3s0)
24/03/28 00:01:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/28 00:01:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

# Question

## List the airlines that operate flights to all available destination

In [14]:
airline_data = [(1, 'Airline A'), (2, 'Airline B'), (3, 'Airline C')]
airline_schema = ['airline_id', 'airline_name']

airline_df = spark.createDataFrame(airline_data, airline_schema)
airline_df.show()

+----------+------------+
|airline_id|airline_name|
+----------+------------+
|         1|   Airline A|
|         2|   Airline B|
|         3|   Airline C|
+----------+------------+



In [11]:
flight_data = [(1,1,101), (2,1,102), (3,2,101), (4,2,103), (5,3,101), (6,3,102), (7,3,103)]
flight_schema = ['flight_id', 'airline_id', 'airport_id']

flight_df = spark.createDataFrame(flight_data, flight_schema)
flight_df.show()

+---------+----------+----------+
|flight_id|airline_id|airport_id|
+---------+----------+----------+
|        1|         1|       101|
|        2|         1|       102|
|        3|         2|       101|
|        4|         2|       103|
|        5|         3|       101|
|        6|         3|       102|
|        7|         3|       103|
+---------+----------+----------+



## Solution

In [12]:
count_airport = flight_df.select('airport_id').distinct().count()
display(count_airport)

3

In [18]:
from pyspark.sql.functions import countDistinct, col

flight_filter_df = flight_df.groupBy('airline_id').agg(countDistinct('airport_id').alias('distinct_airport_count'))
flight_filter_df.show()

+----------+----------------------+
|airline_id|distinct_airport_count|
+----------+----------------------+
|         1|                     2|
|         3|                     3|
|         2|                     2|
+----------+----------------------+



In [19]:
filter_df = flight_filter_df.filter(col('distinct_airport_count') == count_airport)
filter_df.show()

+----------+----------------------+
|airline_id|distinct_airport_count|
+----------+----------------------+
|         3|                     3|
+----------+----------------------+



In [20]:
df_res = filter_df.join(airline_df,['airline_id']).select('airline_id', 'airline_name')
df_res.show()

+----------+------------+
|airline_id|airline_name|
+----------+------------+
|         3|   Airline C|
+----------+------------+

